In [27]:
!pip install -q tensorflow keras numpy pillow h5py torch opencv-python scikit-learn
!pip install -q keras-applications
!pip install -q keras-preprocessing
!pip install -q keras-resnet

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [9]:
import json
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def load_and_preprocess_data(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    metadata = data['metadata']
    action = metadata['inspect']['action']
    
    X = []
    y = []
    
    for annotation in data['annotations']:
        frame_number = annotation['frame_number']
        timestamp = annotation['timestamp']
        image_filename = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
        image_path = os.path.join(image_dir, image_filename)
        
        if not os.path.exists(image_path):
            print(f"이미지 파일이 없습니다: {image_path}")
            continue
        
        frame_keypoints = []
        for i in range(1, 16):
            kp = annotation['keypoints'].get(str(i))
            if kp:
                frame_keypoints.extend([kp['x'], kp['y']])
            else:
                frame_keypoints.extend([0, 0])
        X.append(frame_keypoints)
        y.append(action)
    
    return np.array(X), y, metadata

def prepare_dataset(json_files, image_dirs):
    X_all = []
    y_all = []
    all_metadata = []
    
    for json_file, image_dir in zip(json_files, image_dirs):
        X, y, metadata = load_and_preprocess_data(json_file, image_dir)
        X_all.extend(X)
        y_all.extend(y)
        all_metadata.append(metadata)
    
    X_all = np.array(X_all)
    
    unique_actions = list(set(y_all))
    action_to_index = {action: index for index, action in enumerate(unique_actions)}
    y_encoded = [action_to_index[action] for action in y_all]
    y_categorical = to_categorical(y_encoded)
    
    return X_all, y_categorical, all_metadata, unique_actions

# JSON 파일과 이미지 디렉토리 경로 리스트
json_files = ['E:/LSTN_test/data/ARCH/20201028_cat-arch-000156.mp4.json']
image_dirs = ['E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4']

X, y, metadata, action_classes = prepare_dataset(json_files, image_dirs)

print(f"총 샘플 수: {len(X)}")

if len(X) == 0:
    print("처리할 수 있는 이미지가 없습니다. 프로그램을 종료합니다.")
    exit()

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM 모델 구축
model = Sequential([
    LSTM(64, input_shape=(1, X.shape[1]), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(len(action_classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 입력 데이터 reshape
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {accuracy}")

# 메타데이터 활용 예시
print("\n메타데이터 예시:")
print(json.dumps(metadata[0], indent=2, ensure_ascii=False))

print("\n행동 클래스:")
for i, action in enumerate(action_classes):
    print(f"{i}: {action}")

이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_3_timestamp_200.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_6_timestamp_400.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_9_timestamp_600.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_18_timestamp_1200.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_21_timestamp_1400.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_24_timestamp_1600.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_27_timestamp_1800.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_30_timestamp_2000.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_33_timestamp_2200.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_36_timestamp_2400.jpg
이미지 파일이 없습니다: E:/LSTN_test/data/img/20201028_cat-arch-000156.mp4\frame_39_ti

c:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


c:\ProgramData\anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\keras\src\losses\losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.0000

In [12]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {accuracy}")
# 모델 예측
predictions = model.predict(X_test)
predicted_behaviors = np.argmax(predictions, axis=1)

# 예측 결과 출력
print("\n예측 결과:")
for i, behavior in enumerate(predicted_behaviors):
    print(f"{i}: {behavior}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0000e+00
테스트 정확도: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

예측 결과:
0: 0
1: 0
2: 0
3: 0
4: 0


In [16]:
import torch
import torch.nn as nn
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

def keras_to_pytorch(keras_model):
    class PyTorchLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes):
            super(PyTorchLSTM, self).__init__()
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)
        
        def forward(self, x):
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            out, _ = self.lstm(x, (h0, c0))
            out = self.fc(out[:, -1, :])
            return out

    input_size = keras_model.input_shape[-1]
    hidden_size = keras_model.layers[0].units
    num_layers = sum(1 for layer in keras_model.layers if isinstance(layer, LSTM))
    num_classes = keras_model.output_shape[-1]

    pytorch_model = PyTorchLSTM(input_size, hidden_size, num_layers, num_classes)
    
    # Keras 모델의 가중치를 PyTorch 모델로 복사
    for i, layer in enumerate(keras_model.layers):
        if isinstance(layer, LSTM):
            weights = layer.get_weights()
            if len(weights) == 3:  # Keras LSTM은 보통 3개의 가중치 배열을 가집니다
                # 입력 가중치, 순환 가중치, 편향
                input_weights, recurrent_weights, bias = weights
                pytorch_model.lstm.weight_ih_l0.data = torch.FloatTensor(input_weights.T)
                pytorch_model.lstm.weight_hh_l0.data = torch.FloatTensor(recurrent_weights.T)
                # Keras의 편향은 입력 게이트, 망각 게이트, 셀 게이트, 출력 게이트 순서입니다
                # PyTorch는 입력 게이트, 망각 게이트, 셀 게이트, 출력 게이트 순서입니다
                pytorch_model.lstm.bias_ih_l0.data = torch.FloatTensor(bias[:hidden_size*4])
                pytorch_model.lstm.bias_hh_l0.data = torch.FloatTensor(bias[hidden_size*4:])
        elif isinstance(layer, Dense):
            weights, bias = layer.get_weights()
            pytorch_model.fc.weight.data = torch.FloatTensor(weights.T)
            pytorch_model.fc.bias.data = torch.FloatTensor(bias)

    return pytorch_model

# Keras 모델을 PyTorch 모델로 변환
pytorch_model = keras_to_pytorch(model)

# PyTorch 모델 저장
torch.save(pytorch_model.state_dict(), 'lstm_model.pt')

print("PyTorch 모델이 'lstm_model.pt' 파일로 저장되었습니다.")

PyTorch 모델이 'lstm_model.pt' 파일로 저장되었습니다.


In [22]:
import torch

# 저장된 모델 로드
saved_model = torch.load('lstm_model.pt')

# 모델 상태 확인
print("모델 상태 키:")
for key in saved_model.keys():
    print(key)

print("\n각 레이어의 형태:")
for key, value in saved_model.items():
    print(f"{key}: {value.shape}")

# LSTM 레이어의 입력 및 은닉 상태 크기 확인
if 'lstm.weight_ih_l0' in saved_model:
    input_size = saved_model['lstm.weight_ih_l0'].shape[1]
    hidden_size = saved_model['lstm.weight_ih_l0'].shape[0] // 4
    print(f"\nLSTM 입력 크기: {input_size}")
    print(f"LSTM 은닉 상태 크기: {hidden_size}")

# LSTM 레이어 수 확인
num_layers = sum(1 for key in saved_model if key.startswith('lstm.weight_ih_l'))
print(f"LSTM 레이어 수: {num_layers}")

# 출력 레이어(fc) 크기 확인
if 'fc.weight' in saved_model:
    output_size = saved_model['fc.weight'].shape[0]
    print(f"출력 크기: {output_size}")

모델 상태 키:
lstm.weight_ih_l0
lstm.weight_hh_l0
lstm.bias_ih_l0
lstm.bias_hh_l0
lstm.weight_ih_l1
lstm.weight_hh_l1
lstm.bias_ih_l1
lstm.bias_hh_l1
fc.weight
fc.bias

각 레이어의 형태:
lstm.weight_ih_l0: torch.Size([128, 64])
lstm.weight_hh_l0: torch.Size([128, 32])
lstm.bias_ih_l0: torch.Size([128])
lstm.bias_hh_l0: torch.Size([0])
lstm.weight_ih_l1: torch.Size([256, 64])
lstm.weight_hh_l1: torch.Size([256, 64])
lstm.bias_ih_l1: torch.Size([256])
lstm.bias_hh_l1: torch.Size([256])
fc.weight: torch.Size([1, 16])
fc.bias: torch.Size([1])

LSTM 입력 크기: 64
LSTM 은닉 상태 크기: 32
LSTM 레이어 수: 2
출력 크기: 1


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12192\148980449.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_model = torch.load('lstm_model.pt')


In [2]:
import cv2
import os
import re

# 이미지 파일이 있는 디렉토리 경로
이미지_디렉토리 = r'E:\LSTN_test\data\img\20201028_cat-arch-000156.mp4'

# 출력할 동영상 파일 이름
출력_비디오 = 'test.mp4'

# 프레임 번호를 추출하는 함수
def get_frame_number(filename):
    match = re.search(r'frame_(\d+)_', filename)
    return int(match.group(1)) if match else -1

# 이미지 파일 목록 가져오기 및 정렬
이미지_파일들 = [f for f in os.listdir(이미지_디렉토리) if f.endswith('.jpg') or f.endswith('.png')]
이미지_파일들.sort(key=get_frame_number)  # 프레임 번호 기준으로 정렬

if not 이미지_파일들:
    print("이미지 파일을 찾을 수 없습니다.")
    exit()

# 첫 번째 이미지로 비디오 writer 초기화
첫_이미지_경로 = os.path.join(이미지_디렉토리, 이미지_파일들[0])
첫_이미지 = cv2.imread(첫_이미지_경로)

if 첫_이미지 is None:
    print(f"이미지를 열 수 없습니다: {첫_이미지_경로}")
    exit()

높이, 너비, _ = 첫_이미지.shape
fps = 15  # 초당 프레임 수 설정

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
비디오_writer = cv2.VideoWriter(출력_비디오, fourcc, fps, (너비, 높이))

# 각 이미지를 비디오 프레임으로 추가
for 이미지_파일 in 이미지_파일들:
    이미지_경로 = os.path.join(이미지_디렉토리, 이미지_파일)
    프레임 = cv2.imread(이미지_경로)
    
    if 프레임 is None:
        print(f"이미지를 열 수 없습니다: {이미지_경로}")
        continue
    
    비디오_writer.write(프레임)

# 비디오 writer 해제
비디오_writer.release()

print(f"동영상이 성공적으로 생성되었습니다: {출력_비디오}")

처리된 이미지: frame_0_timestamp_0.jpg
처리된 이미지: frame_12_timestamp_800.jpg
처리된 이미지: frame_15_timestamp_1000.jpg
처리된 이미지: frame_102_timestamp_6800.jpg
처리된 이미지: frame_105_timestamp_7000.jpg
처리된 이미지: frame_108_timestamp_7200.jpg
처리된 이미지: frame_111_timestamp_7400.jpg
처리된 이미지: frame_114_timestamp_7600.jpg
처리된 이미지: frame_117_timestamp_7800.jpg
처리된 이미지: frame_120_timestamp_8000.jpg
처리된 이미지: frame_123_timestamp_8200.jpg
처리된 이미지: frame_126_timestamp_8400.jpg
처리된 이미지: frame_129_timestamp_8600.jpg
처리된 이미지: frame_132_timestamp_8800.jpg
처리된 이미지: frame_135_timestamp_9000.jpg
처리된 이미지: frame_138_timestamp_9200.jpg
처리된 이미지: frame_141_timestamp_9400.jpg
처리된 이미지: frame_144_timestamp_9600.jpg
처리된 이미지: frame_147_timestamp_9800.jpg
처리된 이미지: frame_150_timestamp_10000.jpg
처리된 이미지: frame_153_timestamp_10200.jpg
처리된 이미지: frame_156_timestamp_10400.jpg
처리된 이미지: frame_159_timestamp_10600.jpg
처리된 이미지: frame_162_timestamp_10800.jpg
처리된 이미지: frame_165_timestamp_11000.jpg
동영상이 성공적으로 생성되었습니다: test.mp4
